In [1]:
import datetime

def get_third_wednesday(year, month):
    """Return the third Wednesday of the given month and year."""
    first_day = datetime.date(year, month, 1)
    first_wednesday = first_day + datetime.timedelta(days=(2 - first_day.weekday() + 7) % 7)
    third_wednesday = first_wednesday + datetime.timedelta(weeks=2)
    return third_wednesday

def calculate_imm_dates(date_str):
    """Calculate the next two IMM dates from a given date string 'yyyymmdd'."""
    date = datetime.datetime.strptime(date_str, '%Y%m%d').date()
    imm_months = [3, 6, 9, 12]
    
    # Find the next IMM date
    next_imm_date = None
    for month in imm_months:
        imm_date = get_third_wednesday(date.year, month)
        if imm_date > date:
            next_imm_date = imm_date
            break
    
    # If no IMM date found in the current year, check the next year
    if next_imm_date is None:
        next_imm_date = get_third_wednesday(date.year + 1, imm_months[0])
    
    # Find the second IMM date
    second_imm_date = None
    for month in imm_months:
        imm_date = get_third_wednesday(next_imm_date.year, month)
        if imm_date > next_imm_date:
            second_imm_date = imm_date
            break
    
    # If no second IMM date found in the current year, check the next year
    if second_imm_date is None:
        second_imm_date = get_third_wednesday(next_imm_date.year + 1, imm_months[0])
    
    return next_imm_date, second_imm_date

# Example usage
date_str = '20231015'
imm_date_1, imm_date_2 = calculate_imm_dates(date_str)
print(f"IMM Date 1: {imm_date_1}")
print(f"IMM Date 2: {imm_date_2}")

IMM Date 1: 2023-12-20
IMM Date 2: 2024-03-20


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

# Define Attention Mechanism
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))

    def forward(self, hidden, encoder_outputs):
        # hidden: (batch_size, hidden_size)
        # encoder_outputs: (batch_size, seq_len, hidden_size)

        seq_len = encoder_outputs.size(1)
        hidden = hidden.unsqueeze(1).repeat(1, seq_len, 1)  # (batch_size, seq_len, hidden_size)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))  # (batch_size, seq_len, hidden_size)
        energy = energy.transpose(1, 2)  # (batch_size, hidden_size, seq_len)
        v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)  # (batch_size, 1, hidden_size)
        attention_weights = torch.bmm(v, energy).squeeze(1)  # (batch_size, seq_len)

        return torch.softmax(attention_weights, dim=1)  # (batch_size, seq_len)


# Define Encoder
class LSTMEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(LSTMEncoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)

    def forward(self, x):
        # x: (batch_size, seq_len, input_size)
        outputs, (hidden, cell) = self.lstm(x)
        # outputs: (batch_size, seq_len, hidden_size)
        return outputs, hidden, cell


# Define Decoder with Attention
class LSTMDecoderWithAttention(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers, dropout):
        super(LSTMDecoderWithAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.attention = Attention(hidden_size)
        self.lstm = nn.LSTM(hidden_size + 4, hidden_size, num_layers, batch_first=True, dropout=dropout)  # +4 for time features
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, hidden, cell, encoder_outputs, current_temporal_features):
        # hidden: (num_layers, batch_size, hidden_size)
        # encoder_outputs: (batch_size, seq_len, hidden_size)
        # current_temporal_features: (batch_size, 1, 4)

        attention_weights = self.attention(hidden[-1], encoder_outputs)  # (batch_size, seq_len)
        attention_weights = attention_weights.unsqueeze(1)  # (batch_size, 1, seq_len)
        context = torch.bmm(attention_weights, encoder_outputs)  # (batch_size, 1, hidden_size)

        # Combine context with current temporal features
        decoder_input = torch.cat([context, current_temporal_features], dim=2)  # (batch_size, 1, hidden_size + 4)
        
        # Pass through LSTM
        output, (hidden, cell) = self.lstm(decoder_input, (hidden, cell))
        
        # Pass through the final fully connected layer
        prediction = self.fc(output).squeeze(1)  # (batch_size, output_size)

        return prediction, hidden, cell, attention_weights.squeeze(1)  # Return attention weights


# Define full Encoder-Decoder Model
class LSTMEncoderDecoderWithAttention(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout):
        super(LSTMEncoderDecoderWithAttention, self).__init__()
        self.encoder = LSTMEncoder(input_size, hidden_size, num_layers, dropout)
        self.decoder = LSTMDecoderWithAttention(hidden_size, output_size, num_layers, dropout)

    def forward(self, encoder_input, current_temporal_features):
        # encoder_input: (batch_size, seq_len, input_size)
        # current_temporal_features: (batch_size, 1, 4)

        # Encode input sequence
        encoder_outputs, hidden, cell = self.encoder(encoder_input)

        # Decode using the current temporal features and encoder outputs
        prediction, _, _, attention_weights = self.decoder(hidden, cell, encoder_outputs, current_temporal_features)

        return prediction, attention_weights


# Load and preprocess data
df = pd.read_csv('time_series_data.csv')

# Check for missing values
if df.isnull().sum().sum() > 0:
    df = df.fillna(method='ffill').fillna(method='bfill')

# Define your feature and target columns
features = ['rate_level_1', 'rate_level_2',
            'days_to_end_of_month', 'days_to_ECB_meeting', 'days_to_Fed_meeting', 'ois_sofr_rate']

target = ['rate_level_1', 'rate_level_2']

current_temporal_features = ['days_to_end_of_month', 'days_to_ECB_meeting', 'days_to_Fed_meeting', 'ois_sofr_rate']

# Normalize data
scaler_features = MinMaxScaler()
scaler_target = MinMaxScaler()

# Normalize feature columns
df[features] = scaler_features.fit_transform(df[features])

# Normalize target columns
df[target] = scaler_target.fit_transform(df[target])

# Function to create sequences
def create_sequences(data, target_data, n_timesteps):
    X, y, current_time_features = [], [], []
    for i in range(len(data) - n_timesteps):
        X.append(data[i:i + n_timesteps].values)
        y.append(target_data.iloc[i + n_timesteps].values)
        current_time_features.append(data[current_temporal_features].iloc[i + n_timesteps].values)
    return np.array(X), np.array(y), np.array(current_time_features)

# Prepare sequences
n_timesteps_input = 12
X, y, current_temporal = create_sequences(df[features], df[target], n_timesteps_input)

# Split into train and test sets
X_train, X_test, y_train, y_test, current_temporal_train, current_temporal_test = train_test_split(
    X, y, current_temporal, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)
current_temporal_train = torch.tensor(current_temporal_train, dtype=torch.float32).unsqueeze(1)
current_temporal_test = torch.tensor(current_temporal_test, dtype=torch.float32).unsqueeze(1)

# Hyperparameters
input_size = len(features)  # 6 features
hidden_size = 128
output_size = len(target)  # 2 targets
num_layers = 2
dropout = 0.3
learning_rate = 0.001
n_epochs = 50

# Initialize model
model = LSTMEncoderDecoderWithAttention(input_size, hidden_size, output_size, num_layers, dropout)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

# Training loop
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    output, _ = model(X_train, current_temporal_train)
    
    # Calculate loss
    loss = criterion(output, y_train)
    
    # Backward pass and optimize
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item():.4f}')

# Example evaluation on test set
model.eval()
attention_weights_list = []
with torch.no_grad():
    forecast, attention_weights = model(X_test, current_temporal_test)
    attention_weights_list.append(attention_weights.cpu().numpy())

# Convert list to numpy array
attention_weights_array = np.concatenate(attention_weights_list, axis=0)

# Inverse-transform the predictions to the original scale
forecast_original_scale = scaler_target.inverse_transform(forecast.cpu().numpy())
y_test_original_scale = scaler_target.inverse_transform(y_test.cpu().numpy())

# Compare predictions to the actual values
print("Predictions on original scale:", forecast_original_scale)
print("True values on original scale:", y_test_original_scale)

# Visualize attention weights for a specific sample
sample_index = 0  # Change this to visualize different samples
attention_weights_sample = attention_weights_array[sample_index]

plt.figure(figsize=(10, 6))
sns.heatmap(attention_weights_sample.reshape(1, -1), cmap='viridis', annot=True)
plt.title('Attention Weights for Sample Index {}'.format(sample_index))
plt.xlabel('Input Sequence Index')
plt.ylabel('Attention Weight')
plt.show()

In [ ]:
class Autoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, encoding_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

In [ ]:
# Hyperparameters for the autoencoder
input_dim = len(features)
encoding_dim = 3  # Reduced dimension

# Initialize the autoencoder
autoencoder = Autoencoder(input_dim, encoding_dim)
ae_optimizer = optim.Adam(autoencoder.parameters(), lr=learning_rate)
ae_criterion = nn.MSELoss()

# Prepare data for autoencoder
X_autoencoder = torch.tensor(df[features].values, dtype=torch.float32)

# Training loop for autoencoder
n_epochs_ae = 100
for epoch in range(n_epochs_ae):
    autoencoder.train()
    ae_optimizer.zero_grad()
    
    # Forward pass
    _, decoded = autoencoder(X_autoencoder)
    
    # Calculate loss
    loss = ae_criterion(decoded, X_autoencoder)
    
    # Backward pass and optimize
    loss.backward()
    ae_optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Autoencoder Epoch [{epoch+1}/{n_epochs_ae}], Loss: {loss.item():.4f}')

In [ ]:
# Transform the input features using the trained autoencoder
autoencoder.eval()
with torch.no_grad():
    encoded_features, _ = autoencoder(X_autoencoder)
    df_encoded = pd.DataFrame(encoded_features.numpy(), columns=[f'encoded_{i}' for i in range(encoding_dim)])

# Update the feature columns to use the encoded features
encoded_feature_columns = [f'encoded_{i}' for i in range(encoding_dim)]
df[encoded_feature_columns] = df_encoded

# Function to create sequences with encoded features
def create_sequences(data, target_data, n_timesteps):
    X, y, current_time_features = [], [], []
    for i in range(len(data) - n_timesteps):
        X.append(data[encoded_feature_columns].iloc[i:i + n_timesteps].values)
        y.append(target_data.iloc[i + n_timesteps].values)
        current_time_features.append(data[current_temporal_features].iloc[i + n_timesteps].values)
    return np.array(X), np.array(y), np.array(current_time_features)

# Prepare sequences
n_timesteps_input = 12
X, y, current_temporal = create_sequences(df, df[target], n_timesteps_input)

# Split into train and test sets
X_train, X_test, y_train, y_test, current_temporal_train, current_temporal_test = train_test_split(
    X, y, current_temporal, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)
current_temporal_train = torch.tensor(current_temporal_train, dtype=torch.float32).unsqueeze(1)
current_temporal_test = torch.tensor(current_temporal_test, dtype=torch.float32).unsqueeze(1)

# Hyperparameters for the LSTM model
input_size = encoding_dim  # Reduced dimension
hidden_size = 128
output_size = len(target)  # 2 targets
num_layers = 2
dropout = 0.3
learning_rate = 0.001
n_epochs = 50

# Initialize the LSTM model
model = LSTMEncoderDecoderWithAttention(input_size, hidden_size, output_size, num_layers, dropout)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

# Training loop for the LSTM model
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    output, _ = model(X_train, current_temporal_train)
    
    # Calculate loss
    loss = criterion(output, y_train)
    
    # Backward pass and optimize
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item():.4f}')

# Example evaluation on test set
model.eval()
attention_weights_list = []
with torch.no_grad():
    forecast, attention_weights = model(X_test, current_temporal_test)
    attention_weights_list.append(attention_weights.cpu().numpy())

# Convert list to numpy array
attention_weights_array = np.concatenate(attention_weights_list, axis=0)

# Inverse-transform the predictions to the original scale
forecast_original_scale = scaler_target.inverse_transform(forecast.cpu().numpy())
y_test_original_scale = scaler_target.inverse_transform(y_test.cpu().numpy())

# Compare predictions to the actual values
print("Predictions on original scale:", forecast_original_scale)
print("True values on original scale:", y_test_original_scale)

# Visualize attention weights for a specific sample
sample_index = 0  # Change this to visualize different samples
attention_weights_sample = attention_weights_array[sample_index]

plt.figure(figsize=(10, 6))
sns.heatmap(attention_weights_sample.reshape(1, -1), cmap='viridis', annot=True)
plt.title('Attention Weights for Sample Index {}'.format(sample_index))
plt.xlabel('Input Sequence Index')
plt.ylabel('Attention Weight')
plt.show()

In [ ]:
# Extract encoded features using the trained autoencoder
autoencoder.eval()
with torch.no_grad():
    encoded_features, _ = autoencoder(X_autoencoder)
    df_encoded = pd.DataFrame(encoded_features.numpy(), columns=[f'encoded_{i}' for i in range(encoding_dim)])

In [ ]:
from sklearn.manifold import TSNE

# Apply t-SNE to reduce the dimensionality of the encoded features to 2D
tsne = TSNE(n_components=2, random_state=42)
encoded_features_2d = tsne.fit_transform(df_encoded)

In [ ]:
# Visualize the 2D encoded features
plt.figure(figsize=(10, 6))
plt.scatter(encoded_features_2d[:, 0], encoded_features_2d[:, 1], c='blue', alpha=0.5)
plt.title('2D Visualization of Encoded Features using t-SNE')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.show()

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.manifold import TSNE

# Define the Autoencoder
class Autoencoder(nn.Module):
    def __init__(self, input_dim, encoding_dim):
        super(Autoencoder, self).__init__()
        self.encoder = nn.Sequential(
            nn.Linear(input_dim, 128),
            nn.ReLU(),
            nn.Linear(128, encoding_dim),
            nn.ReLU()
        )
        self.decoder = nn.Sequential(
            nn.Linear(encoding_dim, 128),
            nn.ReLU(),
            nn.Linear(128, input_dim),
            nn.Sigmoid()
        )

    def forward(self, x):
        encoded = self.encoder(x)
        decoded = self.decoder(encoded)
        return encoded, decoded

# Load and preprocess data
df = pd.read_csv('time_series_data.csv')

# Check for missing values
if df.isnull().sum().sum() > 0:
    df = df.fillna(method='ffill').fillna(method='bfill')

# Define your feature and target columns
features = ['rate_level_1', 'rate_level_2',
            'days_to_end_of_month', 'days_to_ECB_meeting', 'days_to_Fed_meeting', 'ois_sofr_rate']

target = ['rate_level_1', 'rate_level_2']

current_temporal_features = ['days_to_end_of_month', 'days_to_ECB_meeting', 'days_to_Fed_meeting', 'ois_sofr_rate']

# Normalize data
scaler_features = MinMaxScaler()
scaler_target = MinMaxScaler()

# Normalize feature columns
df[features] = scaler_features.fit_transform(df[features])

# Normalize target columns
df[target] = scaler_target.fit_transform(df[target])

# Hyperparameters for the autoencoder
input_dim = len(features)
encoding_dim = 3  # Reduced dimension
learning_rate = 0.001

# Initialize the autoencoder
autoencoder = Autoencoder(input_dim, encoding_dim)
ae_optimizer = optim.Adam(autoencoder.parameters(), lr=learning_rate)
ae_criterion = nn.MSELoss()

# Prepare data for autoencoder
X_autoencoder = torch.tensor(df[features].values, dtype=torch.float32)

# Training loop for autoencoder
n_epochs_ae = 100
for epoch in range(n_epochs_ae):
    autoencoder.train()
    ae_optimizer.zero_grad()
    
    # Forward pass
    _, decoded = autoencoder(X_autoencoder)
    
    # Calculate loss
    loss = ae_criterion(decoded, X_autoencoder)
    
    # Backward pass and optimize
    loss.backward()
    ae_optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Autoencoder Epoch [{epoch+1}/{n_epochs_ae}], Loss: {loss.item():.4f}')

# Extract encoded features using the trained autoencoder
autoencoder.eval()
with torch.no_grad():
    encoded_features, _ = autoencoder(X_autoencoder)
    df_encoded = pd.DataFrame(encoded_features.numpy(), columns=[f'encoded_{i}' for i in range(encoding_dim)])

# Apply t-SNE to reduce the dimensionality of the encoded features to 2D
tsne = TSNE(n_components=2, random_state=42)
encoded_features_2d = tsne.fit_transform(df_encoded)

# Visualize the 2D encoded features
plt.figure(figsize=(10, 6))
plt.scatter(encoded_features_2d[:, 0], encoded_features_2d[:, 1], c='blue', alpha=0.5)
plt.title('2D Visualization of Encoded Features using t-SNE')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.show()

In [ ]:
from sklearn.inspection import permutation_importance

# Assuming you have a trained model and test data
result = permutation_importance(model, X_test, y_test, n_repeats=10, random_state=42, n_jobs=-1)

# Get feature importance scores
feature_importance = result.importances_mean

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(range(len(feature_importance)), feature_importance, align='center')
plt.yticks(range(len(feature_importance)), encoded_feature_columns)
plt.xlabel('Permutation Feature Importance')
plt.title('Feature Importance')
plt.show()

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Train a Random Forest model
rf_model = RandomForestRegressor(random_state=42)
rf_model.fit(X_train, y_train)

# Get feature importance scores
feature_importance = rf_model.feature_importances_

# Plot feature importance
plt.figure(figsize=(10, 6))
plt.barh(range(len(feature_importance)), feature_importance, align='center')
plt.yticks(range(len(feature_importance)), encoded_feature_columns)
plt.xlabel('Feature Importance')
plt.title('Feature Importance from Random Forest')
plt.show()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

# Calculate the correlation matrix
correlation_matrix = df[features].corr()

# Plot the heatmap
plt.figure(figsize=(12, 8))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Correlation Heatmap')
plt.show()

In [ ]:
# Plot pairplot
sns.pairplot(df[features])
plt.suptitle('Pairplot of Features', y=1.02)
plt.show()

In [ ]:
# Plot cluster heatmap
sns.clustermap(df[features].corr(), annot=True, cmap='coolwarm', vmin=-1, vmax=1)
plt.title('Cluster Heatmap')
plt.show()

In [ ]:
# Plot feature distribution
plt.figure(figsize=(12, 8))
for i, feature in enumerate(features):
    plt.subplot(2, 3, i + 1)
    sns.histplot(df[feature], kde=True)
    plt.title(f'Distribution of {feature}')
plt.tight_layout()
plt.show()

In [ ]:
# Plot boxplot
plt.figure(figsize=(12, 8))
sns.boxplot(data=df[features])
plt.title('Boxplot of Features')
plt.xticks(rotation=45)
plt.show()

In [ ]:
# Assuming df_encoded contains the encoded features
plt.figure(figsize=(12, 8))
sns.heatmap(df_encoded, cmap='viridis', cbar=True)
plt.title('Heatmap of Encoded Features')
plt.show()

In [ ]:
from sklearn.feature_selection import SelectKBest, chi2

# Select top k features based on chi-square test
k = 5
selector = SelectKBest(chi2, k=k)
X_new = selector.fit_transform(X, y)
selected_features = X.columns[selector.get_support()]
print("Selected features:", selected_features)

In [ ]:
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor

# Initialize the model
model = RandomForestRegressor()

# Initialize RFE with the model
rfe = RFE(model, n_features_to_select=5)
X_rfe = rfe.fit_transform(X, y)
selected_features = X.columns[rfe.support_]
print("Selected features:", selected_features)

In [ ]:
from sklearn.linear_model import Lasso

# Initialize Lasso with a regularization parameter
lasso = Lasso(alpha=0.01)
lasso.fit(X, y)

# Get the coefficients
lasso_coefficients = lasso.coef_

# Select features with non-zero coefficients
selected_features = X.columns[lasso_coefficients != 0]
print("Selected features:", selected_features)

In [ ]:
from sklearn.ensemble import RandomForestRegressor

# Train a Random Forest model
model = RandomForestRegressor(random_state=42)
model.fit(X, y)

# Get feature importance scores
feature_importance = model.feature_importances_

# Select top k features
k = 5
indices = np.argsort(feature_importance)[-k:]
selected_features = X.columns[indices]
print("Selected features:", selected_features)

In [ ]:
from sklearn.feature_selection import mutual_info_regression

# Calculate mutual information scores
mi_scores = mutual_info_regression(X, y)

# Select top k features
k = 5
indices = np.argsort(mi_scores)[-k:]
selected_features = X.columns[indices]
print("Selected features:", selected_features)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import xgboost as xgb
from sklearn.metrics import mean_squared_error

# Load and preprocess data
df = pd.read_csv('time_series_data.csv')

# Check for missing values
if df.isnull().sum().sum() > 0:
    df = df.fillna(method='ffill').fillna(method='bfill')

# Define your feature and target columns
features = ['rate_level_1', 'rate_level_2',
            'days_to_end_of_month', 'days_to_ECB_meeting', 'days_to_Fed_meeting', 'ois_sofr_rate']

target = ['rate_level_1', 'rate_level_2']

# Normalize data
scaler_features = MinMaxScaler()
scaler_target = MinMaxScaler()

# Normalize feature columns
df[features] = scaler_features.fit_transform(df[features])

# Normalize target columns
df[target] = scaler_target.fit_transform(df[target])

# Function to create sequences
def create_sequences(data, target_data, n_timesteps):
    X, y = [], []
    for i in range(len(data) - n_timesteps):
        X.append(data[i:i + n_timesteps].values.flatten())
        y.append(target_data.iloc[i + n_timesteps].values)
    return np.array(X), np.array(y)

# Prepare sequences
n_timesteps_input = 12
X, y = create_sequences(df[features], df[target], n_timesteps_input)

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Initialize XGBoost model
xgb_model = xgb.XGBRegressor(objective='reg:squarederror', n_estimators=100, learning_rate=0.1, max_depth=5, random_state=42)

# Train the model
xgb_model.fit(X_train, y_train)

# Make predictions
y_pred = xgb_model.predict(X_test)

In [ ]:
# Calculate Mean Squared Error
mse = mean_squared_error(y_test, y_pred)
print(f'Mean Squared Error: {mse:.4f}')

# Inverse-transform the predictions to the original scale
y_pred_original_scale = scaler_target.inverse_transform(y_pred.reshape(-1, len(target)))
y_test_original_scale = scaler_target.inverse_transform(y_test)

# Compare predictions to the actual values
print("Predictions on original scale:", y_pred_original_scale)
print("True values on original scale:", y_test_original_scale)

# Visualize predictions vs actual values
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 6))
plt.plot(y_test_original_scale[:, 0], label='True Rate Level 1')
plt.plot(y_pred_original_scale[:, 0], label='Predicted Rate Level 1')
plt.legend()
plt.title('True vs Predicted Rate Level 1')
plt.show()

plt.figure(figsize=(10, 6))
plt.plot(y_test_original_scale[:, 1], label='True Rate Level 2')
plt.plot(y_pred_original_scale[:, 1], label='Predicted Rate Level 2')
plt.legend()
plt.title('True vs Predicted Rate Level 2')
plt.show()

In [ ]:
import shap

# Initialize the SHAP explainer
explainer = shap.Explainer(model, X_train)

# Calculate SHAP values for the test set
shap_values = explainer(X_test)

# Plot SHAP summary plot
shap.summary_plot(shap_values, X_test, feature_names=encoded_feature_columns)

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

# Define Attention Mechanism
class Attention(nn.Module):
    def __init__(self, hidden_size):
        super(Attention, self).__init__()
        self.attn = nn.Linear(hidden_size * 2, hidden_size)
        self.v = nn.Parameter(torch.rand(hidden_size))

    def forward(self, hidden, encoder_outputs):
        # hidden: (batch_size, hidden_size)
        # encoder_outputs: (batch_size, seq_len, hidden_size)

        seq_len = encoder_outputs.size(1)
        hidden = hidden.unsqueeze(1).repeat(1, seq_len, 1)  # (batch_size, seq_len, hidden_size)
        energy = torch.tanh(self.attn(torch.cat((hidden, encoder_outputs), dim=2)))  # (batch_size, seq_len, hidden_size)
        energy = energy.transpose(1, 2)  # (batch_size, hidden_size, seq_len)
        v = self.v.repeat(encoder_outputs.size(0), 1).unsqueeze(1)  # (batch_size, 1, hidden_size)
        attention_weights = torch.bmm(v, energy).squeeze(1)  # (batch_size, seq_len)

        return torch.softmax(attention_weights, dim=1)  # (batch_size, seq_len)


# Define Encoder
class LSTMEncoder(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, dropout):
        super(LSTMEncoder, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True, dropout=dropout)

    def forward(self, x):
        # x: (batch_size, seq_len, input_size)
        outputs, (hidden, cell) = self.lstm(x)
        # outputs: (batch_size, seq_len, hidden_size)
        return outputs, hidden, cell


# Define Decoder with Attention
class LSTMDecoderWithAttention(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers, dropout):
        super(LSTMDecoderWithAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        self.attention = Attention(hidden_size)
        self.lstm = nn.LSTM(hidden_size + 4, hidden_size, num_layers, batch_first=True, dropout=dropout)  # +4 for time features
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, hidden, cell, encoder_outputs, current_temporal_features):
        # hidden: (num_layers, batch_size, hidden_size)
        # encoder_outputs: (batch_size, seq_len, hidden_size)
        # current_temporal_features: (batch_size, 1, 4)

        attention_weights = self.attention(hidden[-1], encoder_outputs)  # (batch_size, seq_len)
        attention_weights = attention_weights.unsqueeze(1)  # (batch_size, 1, seq_len)
        context = torch.bmm(attention_weights, encoder_outputs)  # (batch_size, 1, hidden_size)

        # Combine context with current temporal features
        decoder_input = torch.cat([context, current_temporal_features], dim=2)  # (batch_size, 1, hidden_size + 4)
        
        # Pass through LSTM
        output, (hidden, cell) = self.lstm(decoder_input, (hidden, cell))
        
        # Pass through the final fully connected layer
        prediction = self.fc(output).squeeze(1)  # (batch_size, output_size)

        return prediction, hidden, cell, attention_weights.squeeze(1)  # Return attention weights


# Define full Encoder-Decoder Model
class LSTMEncoderDecoderWithAttention(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, dropout):
        super(LSTMEncoderDecoderWithAttention, self).__init__()
        self.encoder = LSTMEncoder(input_size, hidden_size, num_layers, dropout)
        self.decoder = LSTMDecoderWithAttention(hidden_size, output_size, num_layers, dropout)

    def forward(self, encoder_input, current_temporal_features):
        # encoder_input: (batch_size, seq_len, input_size)
        # current_temporal_features: (batch_size, 1, 4)

        # Encode input sequence
        encoder_outputs, hidden, cell = self.encoder(encoder_input)

        # Decode using the current temporal features and encoder outputs
        prediction, _, _, attention_weights = self.decoder(hidden, cell, encoder_outputs, current_temporal_features)

        return prediction, attention_weights


# Load and preprocess data
df = pd.read_csv('time_series_data.csv')

# Check for missing values
if df.isnull().sum().sum() > 0:
    df = df.fillna(method='ffill').fillna(method='bfill')

# Define your feature and target columns
features = ['rate_level_1', 'rate_level_2',
            'days_to_end_of_month', 'days_to_ECB_meeting', 'days_to_Fed_meeting', 'ois_sofr_rate']

target = ['rate_level_1', 'rate_level_2']

current_temporal_features = ['days_to_end_of_month', 'days_to_ECB_meeting', 'days_to_Fed_meeting', 'ois_sofr_rate']

# Normalize data
scaler_features = MinMaxScaler()
scaler_target = MinMaxScaler()

# Normalize feature columns
df[features] = scaler_features.fit_transform(df[features])

# Normalize target columns
df[target] = scaler_target.fit_transform(df[target])

# Function to create sequences
def create_sequences(data, target_data, n_timesteps):
    X, y, current_time_features = [], [], []
    for i in range(len(data) - n_timesteps):
        X.append(data[i:i + n_timesteps].values)
        y.append(target_data.iloc[i + n_timesteps].values)
        current_time_features.append(data[current_temporal_features].iloc[i + n_timesteps].values)
    return np.array(X), np.array(y), np.array(current_time_features)

# Prepare sequences
n_timesteps_input = 12
X, y, current_temporal = create_sequences(df[features], df[target], n_timesteps_input)

# Split into train and test sets
X_train, X_test, y_train, y_test, current_temporal_train, current_temporal_test = train_test_split(
    X, y, current_temporal, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)
current_temporal_train = torch.tensor(current_temporal_train, dtype=torch.float32).unsqueeze(1)
current_temporal_test = torch.tensor(current_temporal_test, dtype=torch.float32).unsqueeze(1)

# Hyperparameters
input_size = len(features)  # 6 features
hidden_size = 128
output_size = len(target)  # 2 targets
num_layers = 2
dropout = 0.3
learning_rate = 0.001
n_epochs = 50

# Initialize model
model = LSTMEncoderDecoderWithAttention(input_size, hidden_size, output_size, num_layers, dropout)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

# Training loop
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    output, _ = model(X_train, current_temporal_train)
    
    # Calculate loss
    loss = criterion(output, y_train)
    
    # Backward pass and optimize
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item():.4f}')

# Example evaluation on test set
model.eval()
attention_weights_list = []
with torch.no_grad():
    forecast, attention_weights = model(X_test, current_temporal_test)
    attention_weights_list.append(attention_weights.cpu().numpy())

# Convert list to numpy array
attention_weights_array = np.concatenate(attention_weights_list, axis=0)

# Inverse-transform the predictions to the original scale
forecast_original_scale = scaler_target.inverse_transform(forecast.cpu().numpy())
y_test_original_scale = scaler_target.inverse_transform(y_test.cpu().numpy())

# Compare predictions to the actual values
print("Predictions on original scale:", forecast_original_scale)
print("True values on original scale:", y_test_original_scale)

# Visualize attention weights for a specific sample
sample_index = 0  # Change this to visualize different samples
attention_weights_sample = attention_weights_array[sample_index]

plt.figure(figsize=(10, 6))
sns.heatmap(attention_weights_sample.reshape(1, -1), cmap='viridis', annot=True)
plt.title('Attention Weights for Sample Index {}'.format(sample_index))
plt.xlabel('Input Sequence Index')
plt.ylabel('Attention Weight')
plt.show()

In [ ]:
def create_sequences(data, target_data, n_timesteps, n_future):
    X, y, current_time_features = [], [], []
    for i in range(len(data) - n_timesteps - n_future + 1):
        X.append(data[i:i + n_timesteps].values)
        y.append(target_data.iloc[i + n_timesteps:i + n_timesteps + n_future].values)
        current_time_features.append(data[current_temporal_features].iloc[i + n_timesteps:i + n_timesteps + n_future].values)
    return np.array(X), np.array(y), np.array(current_time_features)

In [ ]:
class LSTMDecoderWithAttention(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers, dropout, n_future):
        super(LSTMDecoderWithAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.n_future = n_future

        self.attention = Attention(hidden_size)
        self.lstm = nn.LSTM(hidden_size + 4, hidden_size, num_layers, batch_first=True, dropout=dropout)  # +4 for time features
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, hidden, cell, encoder_outputs, current_temporal_features):
        # hidden: (num_layers, batch_size, hidden_size)
        # encoder_outputs: (batch_size, seq_len, hidden_size)
        # current_temporal_features: (batch_size, n_future, 4)

        predictions = []
        attention_weights_list = []

        for t in range(self.n_future):
            attention_weights = self.attention(hidden[-1], encoder_outputs)  # (batch_size, seq_len)
            attention_weights = attention_weights.unsqueeze(1)  # (batch_size, 1, seq_len)
            context = torch.bmm(attention_weights, encoder_outputs)  # (batch_size, 1, hidden_size)

            # Combine context with current temporal features
            decoder_input = torch.cat([context, current_temporal_features[:, t:t+1, :]], dim=2)  # (batch_size, 1, hidden_size + 4)
            
            # Pass through LSTM
            output, (hidden, cell) = self.lstm(decoder_input, (hidden, cell))
            
            # Pass through the final fully connected layer
            prediction = self.fc(output).squeeze(1)  # (batch_size, output_size)
            predictions.append(prediction)
            attention_weights_list.append(attention_weights.squeeze(1))

        predictions = torch.stack(predictions, dim=1)  # (batch_size, n_future, output_size)
        attention_weights_list = torch.stack(attention_weights_list, dim=1)  # (batch_size, n_future, seq_len)

        return predictions, hidden, cell, attention_weights_list  # Return attention weights

In [ ]:
# Prepare sequences
n_timesteps_input = 12
n_future = 5
X, y, current_temporal = create_sequences(df[features], df[target], n_timesteps_input, n_future)

# Split into train and test sets
X_train, X_test, y_train, y_test, current_temporal_train, current_temporal_test = train_test_split(
    X, y, current_temporal, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)
current_temporal_train = torch.tensor(current_temporal_train, dtype=torch.float32)
current_temporal_test = torch.tensor(current_temporal_test, dtype=torch.float32)

# Hyperparameters
input_size = len(features)  # 6 features
hidden_size = 128
output_size = len(target)  # 2 targets
num_layers = 2
dropout = 0.3
learning_rate = 0.001
n_epochs = 50

# Initialize model
model = LSTMEncoderDecoderWithAttention(input_size, hidden_size, output_size, num_layers, dropout, n_future)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

# Training loop
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    output, _ = model(X_train, current_temporal_train)
    
    # Calculate loss
    loss = criterion(output, y_train)
    
    # Backward pass and optimize
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item():.4f}')

# Example evaluation on test set
model.eval()
attention_weights_list = []
with torch.no_grad():
    forecast, attention_weights = model(X_test, current_temporal_test)
    attention_weights_list.append(attention_weights.cpu().numpy())

# Convert list to numpy array
attention_weights_array = np.concatenate(attention_weights_list, axis=0)

# Inverse-transform the predictions to the original scale
forecast_original_scale = scaler_target.inverse_transform(forecast.cpu().numpy().reshape(-1, forecast.shape[-1])).reshape(forecast.shape)
y_test_original_scale = scaler_target.inverse_transform(y_test.cpu().numpy().reshape(-1, y_test.shape[-1])).reshape(y_test.shape)

# Compare predictions to the actual values
print("Predictions on original scale:", forecast_original_scale)
print("True values on original scale:", y_test_original_scale)

# Visualize attention weights for a specific sample
sample_index = 0  # Change this to visualize different samples
attention_weights_sample = attention_weights_array[sample_index]

plt.figure(figsize=(10, 6))
sns.heatmap(attention_weights_sample, cmap='viridis', annot=True)
plt.title('Attention Weights for Sample Index {}'.format(sample_index))
plt.xlabel('Input Sequence Index')
plt.ylabel('Attention Weight')
plt.show()

In [ ]:
def create_sequences(data, target_data, n_timesteps, n_future):
    X, y, current_time_features = [], [], []
    for i in range(len(data) - n_timesteps - n_future + 1):
        X.append(data[i:i + n_timesteps].values)
        y.append(target_data.iloc[i + n_timesteps:i + n_timesteps + n_future].values)
        current_time_features.append(data[current_temporal_features].iloc[i + n_timesteps:i + n_timesteps + n_future].values)
    return np.array(X), np.array(y), np.array(current_time_features)

In [ ]:
predictable_features = ['rate_level_1', 'rate_level_2', 'days_to_ECB_meeting', 'days_to_Fed_meeting', 'ois_sofr_rate']
non_predictable_features = ['days_to_end_of_month']

In [ ]:
def create_sequences(data, target_data, n_timesteps, n_future):
    X, y, current_time_features, non_predictable_time_features = [], [], [], []
    for i in range(len(data) - n_timesteps - n_future + 1):
        X.append(data[predictable_features].iloc[i:i + n_timesteps].values)
        y.append(target_data.iloc[i + n_timesteps:i + n_timesteps + n_future].values)
        current_time_features.append(data[predictable_features].iloc[i + n_timesteps:i + n_timesteps + n_future].values)
        non_predictable_time_features.append(data[non_predictable_features].iloc[i + n_timesteps:i + n_timesteps + n_future].values)
    return np.array(X), np.array(y), np.array(current_time_features), np.array(non_predictable_time_features)

In [ ]:
class LSTMDecoderWithAttention(nn.Module):
    def __init__(self, hidden_size, output_size, num_layers, dropout, n_future):
        super(LSTMDecoderWithAttention, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.n_future = n_future

        self.attention = Attention(hidden_size)
        self.lstm = nn.LSTM(hidden_size + len(predictable_features) + len(non_predictable_features), hidden_size, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, hidden, cell, encoder_outputs, current_temporal_features, non_predictable_features):
        # hidden: (num_layers, batch_size, hidden_size)
        # encoder_outputs: (batch_size, seq_len, hidden_size)
        # current_temporal_features: (batch_size, n_future, len(predictable_features))
        # non_predictable_features: (batch_size, n_future, len(non_predictable_features))

        predictions = []
        attention_weights_list = []

        for t in range(self.n_future):
            attention_weights = self.attention(hidden[-1], encoder_outputs)  # (batch_size, seq_len)
            attention_weights = attention_weights.unsqueeze(1)  # (batch_size, 1, seq_len)
            context = torch.bmm(attention_weights, encoder_outputs)  # (batch_size, 1, hidden_size)

            # Combine context with current temporal features and non-predictable features
            decoder_input = torch.cat([context, current_temporal_features[:, t:t+1, :], non_predictable_features[:, t:t+1, :]], dim=2)  # (batch_size, 1, hidden_size + len(predictable_features) + len(non_predictable_features))
            
            # Pass through LSTM
            output, (hidden, cell) = self.lstm(decoder_input, (hidden, cell))
            
            # Pass through the final fully connected layer
            prediction = self.fc(output).squeeze(1)  # (batch_size, output_size)
            predictions.append(prediction)
            attention_weights_list.append(attention_weights.squeeze(1))

        predictions = torch.stack(predictions, dim=1)  # (batch_size, n_future, output_size)
        attention_weights_list = torch.stack(attention_weights_list, dim=1)  # (batch_size, n_future, seq_len)

        return predictions, hidden, cell, attention_weights_list  # Return attention weights

In [ ]:
# Prepare sequences
n_timesteps_input = 12
n_future = 5
X, y, current_temporal, non_predictable_temporal = create_sequences(df, df[target], n_timesteps_input, n_future)

# Split into train and test sets
X_train, X_test, y_train, y_test, current_temporal_train, current_temporal_test, non_predictable_temporal_train, non_predictable_temporal_test = train_test_split(
    X, y, current_temporal, non_predictable_temporal, test_size=0.2, random_state=42)

# Convert to PyTorch tensors
X_train = torch.tensor(X_train, dtype=torch.float32)
X_test = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)
current_temporal_train = torch.tensor(current_temporal_train, dtype=torch.float32)
current_temporal_test = torch.tensor(current_temporal_test, dtype=torch.float32)
non_predictable_temporal_train = torch.tensor(non_predictable_temporal_train, dtype=torch.float32)
non_predictable_temporal_test = torch.tensor(non_predictable_temporal_test, dtype=torch.float32)

# Hyperparameters
input_size = len(predictable_features)  # 5 features
hidden_size = 128
output_size = len(target)  # 2 targets
num_layers = 2
dropout = 0.3
learning_rate = 0.001
n_epochs = 50

# Initialize model
model = LSTMEncoderDecoderWithAttention(input_size, hidden_size, output_size, num_layers, dropout, n_future)
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.MSELoss()

# Training loop
for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    output, _ = model(X_train, current_temporal_train, non_predictable_temporal_train)
    
    # Calculate loss
    loss = criterion(output, y_train)
    
    # Backward pass and optimize
    loss.backward()
    optimizer.step()

    if (epoch + 1) % 10 == 0:
        print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item():.4f}')

# Example evaluation on test set
model.eval()
attention_weights_list = []
with torch.no_grad():
    forecast, attention_weights = model(X_test, current_temporal_test, non_predictable_temporal_test)
    attention_weights_list.append(attention_weights.cpu().numpy())

# Convert list to numpy array
attention_weights_array = np.concatenate(attention_weights_list, axis=0)

# Inverse-transform the predictions to the original scale
forecast_original_scale = scaler_target.inverse_transform(forecast.cpu().numpy().reshape(-1, forecast.shape[-1])).reshape(forecast.shape)
y_test_original_scale = scaler_target.inverse_transform(y_test.cpu().numpy().reshape(-1, y_test.shape[-1])).reshape(y_test.shape)

# Compare predictions to the actual values
print("Predictions on original scale:", forecast_original_scale)
print("True values on original scale:", y_test_original_scale)

# Visualize attention weights for a specific sample
sample_index = 0  # Change this to visualize different samples
attention_weights_sample = attention_weights_array[sample_index]

plt.figure(figsize=(10, 6))
sns.heatmap(attention_weights_sample, cmap='viridis', annot=True)
plt.title('Attention Weights for Sample Index {}'.format(sample_index))
plt.xlabel('Input Sequence Index')
plt.ylabel('Attention Weight')
plt.show()

In [ ]:
import pandas as pd
import numpy as np

# Sample list of dates
dates = ['2023-01-15', '2023-02-20', '2023-03-25', '2023-04-30']

# Convert to DataFrame
df = pd.DataFrame({'date': pd.to_datetime(dates)})

# Extract month as 'MMM'
df['month'] = df['date'].dt.strftime('%b')

# One-Hot Encoding
one_hot_encoded = pd.get_dummies(df['month'])

# Ordinal Encoding
ordinal_mapping = {month: i for i, month in enumerate(['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec'], 1)}
df['month_ordinal'] = df['month'].map(ordinal_mapping)

# Sine and Cosine Encoding
df['month_num'] = df['date'].dt.month
df['month_sin'] = np.sin(2 * np.pi * df['month_num'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month_num'] / 12)

print("One-Hot Encoded:\n", one_hot_encoded)
print("Ordinal Encoded:\n", df[['month', 'month_ordinal']])
print("Sine and Cosine Encoded:\n", df[['month', 'month_sin', 'month_cos']])

In [ ]:
One-Hot Encoding:

This method creates a binary vector for each month. For example, January would be [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], February would be [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], and so on.
Ordinal Encoding:

This method assigns an integer to each month. For example, January would be 1, February would be 2, and so on.
Sine and Cosine Encoding:

This method encodes the month as two continuous features using sine and cosine functions. This is useful for capturing the cyclical nature of months.

In [ ]:
import pandas as pd
import numpy as np

# Sample data
dates = ['2021-01-01', '2021-02-01', '2021-03-01', '2021-04-01', '2021-05-01', '2021-06-01',
         '2021-07-01', '2021-08-01', '2021-09-01', '2021-10-01', '2021-11-01', '2021-12-01']

# Convert to DataFrame
df = pd.DataFrame({'date': pd.to_datetime(dates)})

# Extract month number
df['month_num'] = df['date'].dt.month

# Sine and Cosine Encoding
df['month_sin'] = np.sin(2 * np.pi * df['month_num'] / 12)
df['month_cos'] = np.cos(2 * np.pi * df['month_num'] / 12)

print("Sine and Cosine Encoded:\n", df[['date', 'month_sin', 'month_cos']])

In [30]:
import pandas as pd

# Sample data
data = {
    'cut_id': ['2021-10-01', '2021-10-01', '2021-10-01', '2021-10-01', '2021-10-01', '2021-10-01', '2021-10-01', '2021-10-01', '2021-10-01',
               '2021-10-04', '2021-10-04', '2021-10-04', '2021-10-04', '2021-10-04', '2021-10-04', '2021-10-04', '2021-10-04', '2021-10-04',
               '2021-10-05', '2021-10-05', '2021-10-05', '2021-10-05', '2021-10-05'],
    'name': ["b'USD Stub'", "b'USD 2021-11-03'", "b'USD 2021-12-01'", "b'USD 2022-01-03'", "b'USD 2022-02-01'", "b'USD 2022-03-01'", "b'USD 2022-04-01'", "b'USD 2022-05-02'", "b'USD 2022-06-01'",
             "b'USD Stub'", "b'USD 2021-11-03'", "b'USD 2021-12-01'", "b'USD 2022-01-03'", "b'USD 2022-02-01'", "b'USD 2022-03-01'", "b'USD 2022-04-01'", "b'USD 2022-05-02'", "b'USD 2022-06-01'",
             "b'USD Stub'", "b'USD 2021-11-03'", "b'USD 2021-12-01'", "b'USD 2022-01-03'", "b'USD 2022-02-01'"],
    'rate': [0.0001, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008, 0.0009,
             0.0005, 0.0002, 0.0003, 0.0004, 0.0005, 0.0006, 0.0007, 0.0008, 0.0009,
             0.0005, 0.0002, 0.0003, 0.0004, 0.0005]
}

# Convert to DataFrame
df = pd.DataFrame(data)

# Function to extract rates and meeting dates
def extract_meeting_info(df):
    # Ensure the date columns are in datetime format
    df['cut_id'] = pd.to_datetime(df['cut_id'])

    # Function to parse meeting dates
    def parse_meeting_date(name):
        try:
            # Extract the date part from the string
            date_str = name.split()[1][0:-1]
            return pd.to_datetime(date_str)
        except Exception as e:
            return None

    # Extract meeting dates
    df['meeting_date'] = df['name'].apply(lambda x: parse_meeting_date(x) if 'USD' in x else None)

    # Ensure meeting_date column is in datetime format
    df['meeting_date'] = pd.to_datetime(df['meeting_date'], errors='coerce')

    # Initialize lists to store the results
    stub_rate_list = []
    first_meeting_days_list = []
    first_meeting_rate_list = []
    second_meeting_days_list = []
    second_meeting_rate_list = []

    # Get unique cut_ids
    unique_cut_ids = df['cut_id'].unique()

    # Iterate over each unique cut_id
    for cut_id in unique_cut_ids:
        # Filter the DataFrame for the current cut_id
        current_df = df[df['cut_id'] == cut_id]

        # Extract the stub rate
        stub_rate = current_df[current_df['name'] == "b'USD Stub'"]['rate'].values[0]

        # Extract the first and second meeting dates and rates
        meeting_dates = current_df[current_df['name'] != "b'USD Stub'"].sort_values(by='meeting_date')
        if len(meeting_dates) >= 2:
            first_meeting_days = (meeting_dates.iloc[0]['meeting_date'] - cut_id).days
            first_meeting_rate = meeting_dates.iloc[0]['rate']
            second_meeting_days = (meeting_dates.iloc[1]['meeting_date'] - cut_id).days
            second_meeting_rate = meeting_dates.iloc[1]['rate']
        elif len(meeting_dates) == 1:
            first_meeting_days = (meeting_dates.iloc[0]['meeting_date'] - cut_id).days
            first_meeting_rate = meeting_dates.iloc[0]['rate']
            second_meeting_days = None
            second_meeting_rate = None
        else:
            first_meeting_days = None
            first_meeting_rate = None
            second_meeting_days = None
            second_meeting_rate = None

        # Append the results to the lists
        stub_rate_list.append(stub_rate)
        first_meeting_days_list.append(first_meeting_days)
        first_meeting_rate_list.append(first_meeting_rate)
        second_meeting_days_list.append(second_meeting_days)
        second_meeting_rate_list.append(second_meeting_rate)

    # Create a new DataFrame with the results
    result_df = pd.DataFrame({
        'cut_id': unique_cut_ids,
        'stub_rate': stub_rate_list,
        'first_meeting_days': first_meeting_days_list,
        'first_meeting_rate': first_meeting_rate_list,
        'second_meeting_days': second_meeting_days_list,
        'second_meeting_rate': second_meeting_rate_list
    })

    return result_df

# Extract meeting info
result_df = extract_meeting_info(df)
print(result_df)

      cut_id  stub_rate  first_meeting_days  first_meeting_rate  \
0 2021-10-01     0.0001                  33              0.0002   
1 2021-10-04     0.0005                  30              0.0002   
2 2021-10-05     0.0005                  29              0.0002   

   second_meeting_days  second_meeting_rate  
0                   61               0.0003  
1                   58               0.0003  
2                   57               0.0003  
